In [1]:
import pandas as pd
import sys
import datetime
import numpy as np

sys.path.append("/home/tetotille/Escritorio/MOPC")
from utils.functions_ia import tipo_doc_1, standard_dni, dni_characters, cuit,dni_zero, tipo_doc_2

# Para enviar a certero

In [8]:
df = pd.read_csv("Datos_Enriquecimiento.csv")
df = df.drop(["ESTADO","IDCLIENTE","DATOADIC","FILTROA"],axis=1)
df = df.reindex(columns=["IDMOROSO","DOCUMENTO","NOMBRE","TIPO"])
df.to_csv("A_CERTERO.csv",index=False)
df.to_excel("A_CERTERO.xlsx",index=False)


# Preprocesamiento de los datos de Certero
Anexamos los datos que nos dió el proveedor para luego poder compararlos

In [261]:
df = pd.read_csv("Datos_Enriquecimiento.csv")
df["DATO"]=None

dni = df[df["TIPO"]=="DNI"]
dni.loc[:,("DOCUMENTO")]=dni.loc[:,("DOCUMENTO")].astype(np.int64)
dni = dni.drop(["Unnamed: 8","Unnamed: 9"],axis=1)

certero = pd.read_csv("sumario.csv")
certero = certero.fillna(0)

#Left Join
#pd.merge(dni,certero,on='DOCUMENTO',how='left')

filtro = [x in certero["DOCUMENTO"].tolist() for x in dni["DOCUMENTO"].tolist()]

print("Total de la base de datos:",len(dni))
print("Datos encontrados en la base de datos:",sum(filtro))
dni[filtro]

Total de la base de datos: 19602
Datos encontrados en la base de datos: 5


/home/tetotille/miniconda3/envs/mopc/lib/python3.7/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


,IDMOROSO,ESTADO,IDCLIENTE,DATOADIC,NOMBRE,FILTROA,DOCUMENTO,TIPO,DATO
40,DNI 12552667,NO UBICADO,BSR_TARDIA,,SANCHEZ MARIO JOSE,Campana Extrajud Ind,12552667,DNI,None
5000,DNI 19060653,ASIGNADO,FRANCES_2012,,FERNANDEZ REBECA NOEMI,3 - Ag 3ra,19060653,DNI,None
6513,DNI 16529147,INTERESADO_PORTAL,FRANCES_2012,,OLIVA MARCELO AGUSTIN,3 - Ag 3ra,16529147,DNI,None
6916,DNI 14002409,ASIGNADO,FRANCES_2012,,CARMONA CARLOS RUBEN,1 - Ag 1ra,14002409,DNI,None
13530,DNI 22281497,NO INTERESADO,FRANCES_2012,,GONZALEZ MIRIAN MABEL,1 - Ag 1ra,22281497,DNI,None


In [272]:
a = [x in dni["DOCUMENTO"].tolist() for x in certero["DOCUMENTO"].tolist()]
certero[a]    

,#ID,DOCUMENTO,CUIL,APELLIDO,NOMBRE,SEXO,FECHA_NAC,DIFUNTO,PADRON_DOMICILIO,PADRON_LOCALIDAD,...,TELEFONO_FIJO1,TELEFONO_FIJO1_OPERADOR,TELEFONO_FIJO2,TELEFONO_FIJO2_OPERADOR,TELEFONO_FIJO3,TELEFONO_FIJO3_OPERADOR,TELEFONO_FIJO4,TELEFONO_FIJO4_OPERADOR,EMAIL,EMAIL2
0,0.0,14002409,20140024091,CARMONA,CARLOS RUBEN,M,19/06/1961,NO,UJEMVI,EL ZAPALLAR,...,0.000000e+00,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
2,0.0,16529147,20165291477,OLIVA,MARCELO AGUSTIN,M,30/04/1963,NO,CASTELLI 1605,BILLINGHURST,...,1.147529e+09,Telecom Argentina,0.0,0.0,0.0,0.0,0.0,0.0,MARCELO_A_OLIVA@HOTMAIL.COM,0
3,0.0,19060653,27190606533,FERNANDEZ REBECA NOEMI,NOEMI,F,01/01/0001,NO,0,0,...,0.000000e+00,0,0.0,0.0,0.0,0.0,0.0,0.0,REBEF@GMAIL.COM,0
4,0.0,22281497,27222814974,GONZALEZ,MIRIAN MABEL,F,04/03/1972,NO,LAS PIEDRAS 364,RIO GALLEGOS,...,0.000000e+00,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
5,0.0,12552667,27190606533,TILLERIA,JORGE,M,15/03/97,NO,asdfasd,oapisjfdio,...,0.000000e+00,0,0.0,0.0,0.0,0.0,0.0,0.0,tetotille@gmail.com,jorge.tilleria@mopc.cc


In [251]:
def obtenerDatos(fila_temp, datos):
    lista = []
    if datos["TELEFONO_FIJO1"].empty:
        fila_temp["DATO"] = datos["TELEFONO_FIJO1"]
        lista.append(fila_temp.copy())
    
    if datos["TELEFONO_FIJO2"].empty:
        fila_temp["DATO"] = datos["TELEFONO_FIJO2"]
        lista.append(fila_temp.copy())
    
    if datos["TELEFONO_FIJO3"].empty:
        fila_temp["DATO"] = datos["TELEFONO_FIJO3"]
        lista.append(fila_temp.copy())
    
    if datos["TELEFONO_FIJO4"].empty:
        fila_temp["DATO"] = datos["TELEFONO_FIJO4"]
        lista.append(fila_temp.copy())
    
    if datos["EMAIL"].empty:
        fila_temp["DATO"] = datos["EMAIL"]
        lista.append(fila_temp.copy())
    
    if datos["EMAIL2"].empty:
        fila_temp["DATO"] = datos["EMAIL2"]
        lista.append(fila_temp.copy())
    return lista

# ACÁ SE CONTINUA
Lo que hay que hacer es que en cada fila que haya dato en certero se le agrega el dato a la fila de nuestro dni, guardando copias y demás y luego eliminando las filas originales sin dato.

In [273]:
dni_original = dni.copy()
a=0
lista_indices=[]
for idx, x in dni_original.iterrows():
    datos = certero.loc[certero["DOCUMENTO"]==x["DOCUMENTO"]]
    if datos.empty: continue
    print("hola")
    print(datos)
    fila = x.copy()
    lista_indices.append(idx)
    
    #quitar y poner los nuevos datos
    lista_datos = obtenerDatos(fila,datos)
    print(lista_datos)
    for dato_nuevo in lista_datos:
        dni = dni.append(dato_nuevo)
    a+=len(lista_datos)

for idx in lista_indices:
    dni = dni.drop(index=idx)
    
print("Se agregaron",a,"datos de contacto")
dni.head()

hola
   #ID  DOCUMENTO         CUIL  APELLIDO NOMBRE SEXO FECHA_NAC DIFUNTO  \
5  0.0   12552667  27190606533  TILLERIA  JORGE    M  15/03/97      NO   

  PADRON_DOMICILIO PADRON_LOCALIDAD  ... TELEFONO_FIJO1  \
5          asdfasd       oapisjfdio  ...            0.0   

   TELEFONO_FIJO1_OPERADOR  TELEFONO_FIJO2  TELEFONO_FIJO2_OPERADOR  \
5                        0             0.0                      0.0   

   TELEFONO_FIJO3  TELEFONO_FIJO3_OPERADOR TELEFONO_FIJO4  \
5             0.0                      0.0            0.0   

   TELEFONO_FIJO4_OPERADOR                EMAIL                  EMAIL2  
5                      0.0  tetotille@gmail.com  jorge.tilleria@mopc.cc  

[1 rows x 33 columns]
   #ID  DOCUMENTO         CUIL  APELLIDO NOMBRE SEXO FECHA_NAC DIFUNTO  \
5  0.0   12552667  27190606533  TILLERIA  JORGE    M  15/03/97      NO   

  PADRON_DOMICILIO PADRON_LOCALIDAD  ... TELEFONO_FIJO1  \
5          asdfasd       oapisjfdio  ...            0.0   

   TELEFONO_FIJO1_OPER

,IDMOROSO,ESTADO,IDCLIENTE,DATOADIC,NOMBRE,FILTROA,DOCUMENTO,TIPO,DATO
14,DNI 12254456,NO UBICADO,BSR_TARDIA,,MASMUK JULIO ARMANDO,Campana Extrajud Ind,12254456,DNI,None
15,DNI 12254927,NO UBICADO,BSR_TARDIA,,GONZALEZ JORGE AVELINO,Campana Extrajud Ind,12254927,DNI,None
16,DNI 12261589,NO UBICADO,BSR_TARDIA,,"MARTHA,ESCALANTE",Campana Extrajud Ind,12261589,DNI,None
17,DNI 12279033,INTERESADO,BSR_TARDIA,,MANUELI JORGE OMAR,Campana Extrajud Ind,12279033,DNI,None
18,DNI 12299611,NO INTERESADO,BSR_TARDIA,,MUNOZ MIGUEL EDUARDO,Campana Extrajud Ind,12299611,DNI,None


In [231]:
for i in range(10):
    if i==2: continue
    print(i)

0
1
3
4
5
6
7
8
9


In [2]:
def procesar(df):
    df.columns = [c.upper().strip() for c in df.columns] # TODO QUIERO EN MAYÚSCULA

    df['IDMOROSO'] = df['IDMOROSO'].str.strip()

    df.loc[(df['IDCLIENTE'].str.startswith('AMEX')) , 'NRO_DOC'] = df['DATOADIC']
    df.loc[~(df['IDCLIENTE'].str.startswith('AMEX')) , 'NRO_DOC'] = df['IDMOROSO']

    df = df.dropna(subset=['NRO_DOC'])

    df['NRO_DOC']=df['NRO_DOC'].str.strip()

    #implementamos las funciones de limpieza de documento
    df['TIPO_DOC'] = df['NRO_DOC'].map(tipo_doc_1)
    df['NRO_DOC'] = df['NRO_DOC'].map(standard_dni)
    df['NRO_DOC'] = df['NRO_DOC'].map(dni_characters)
    df['NRO_DOC'] = df['NRO_DOC'].map(cuit)
    df['NRO_DOC'] = df['NRO_DOC'].map(dni_zero)

    #Para corregir el TIPO DE DOCUMENTO sobre el dataframe resultante
    df['TIPO_DOC'] = df.apply(lambda x: tipo_doc_2(x.NRO_DOC, x.TIPO_DOC), axis=1)

    #Acá hay que eliminar los duplicados
    df = df.drop_duplicates()
    return df

In [3]:
df = pd.read_csv("_Para_comparar_si_es_que_tenemos_ya_algún_dato_de_la_persona_De__202108191747.csv")
df = procesar(df)

#export
df_export = df.rename(columns={'NRO_DOC':'DOCUMENTO','TIPO_DOC':'TIPO', 'NOMBRE':'NOMBRE'})


filename_envio ="Datos_Enriquecimiento.xlsx".format(datetime.datetime.now().strftime('%d%m%y'))
df_export.to_excel(filename_envio, index = None)

In [5]:
df.drop_duplicates()

,IDMOROSO,IDCLIENTE,TIPO_DOC,NRO_DOC,DATOADIC
0,RMS376635149771007,AMEX_ALHR,DNI,4839229,20048392297
2,RMS376635248871005,AMEX_ALHR,DNI,13906211,20139062117
3,RMS376635559571004,AMEX_ALHR,DNI,28679446,27286794462
6,RMS376635612543008,AMEX_ALHR,DNI,26800469,20268004697
8,RMS376635670991008,AMEX_ALHR,DNI,12894123,20128941232
...,...,...,...,...,...
45249,PAS 18348049,FRANCES_2012,PAS,18348049,
45250,PAS 200576017,FRANCES_2012,PAS,57601,CANCELACIÓN
45251,PAS 243586,BSR_TARDIA,PAS,243586,20604590095
45253,PAS 33840,FRANCES_2012,PAS,33840,


In [4]:
proveedor = pd.read_csv("tabla_definitiva_IX.csv")
proveedor = procesar(proveedor)
proveedor = proveedor.drop(['IDMOROSO','IDCLIENTE','DATOADIC','SCORE_DATO','TIENE_MAIL','TIENE_TELEFONO'],axis=1)
proveedor.head()

,TIPO_DOC,NRO_DOC,TIPO,DATO
0,DNI,35399142,TEL.CELULAR,(11) 3350-2328
1,DNI,35399142,TEL.CELULAR,(11) 3447-0813
2,DNI,35399142,TEL.FIJO,(11) 4450-8683
3,DNI,35399142,TEL.CELULAR,(11) 6256-1308
4,DNI,35399142,EMAIL,vanesa.biz@gmail.com


# Acá se tienen todos los datos

In [7]:
data = pd.read_csv("tabla_definitiva_NO_IX.csv")
data = procesar(data)
data.head()

,IDMOROSO,IDCLIENTE,TIPO_DOC,NRO_DOC,DATOADIC,TIPO,DATO,SCORE_DATO,TIENE_MAIL,TIENE_TELEFONO
0,RMS376635149771007,AMEX_ALHR,DNI,4839229,20048392297,TEL.FIJO,1142463085,50.0,1,1
1,RMS376635149771007,AMEX_ALHR,DNI,4839229,20048392297,TEL.FIJO,1142894050,45.0,1,1
2,RMS376635149771007,AMEX_ALHR,DNI,4839229,20048392297,TEL.CELULAR,1160929760,50.0,1,1
3,RMS376635149771007,AMEX_ALHR,DNI,4839229,20048392297,TEL.FIJO,43716273,60.0,1,1
4,RMS376635149771007,AMEX_ALHR,DNI,4839229,20048392297,EMAIL,copybar@speedy.com.ar,33.0,1,1


# ¿Qué lo que yo quiero hacer?

<br>
<font size=3>


Bueno primero seguro que quiero comparar si

<b>

* Hay o no hay?
    
</b>
</font>

In [8]:
data = data.drop(['IDMOROSO','IDCLIENTE', 'DATOADIC','SCORE_DATO', 'TIPO','DATO'],axis=1)
data = data.drop_duplicates()

In [9]:
data.head()

,TIPO_DOC,NRO_DOC,TIENE_MAIL,TIENE_TELEFONO
0,DNI,4839229,1,1
6,DNI,13906211,0,1
8,DNI,28679446,1,1
18,DNI,26800469,1,1
30,DNI,12894123,1,1


In [16]:
proveedor.head()

,TIPO_DOC,NRO_DOC,TIPO,DATO
0,DNI,35399142,TEL.CELULAR,(11) 3350-2328
1,DNI,35399142,TEL.CELULAR,(11) 3447-0813
2,DNI,35399142,TEL.FIJO,(11) 4450-8683
3,DNI,35399142,TEL.CELULAR,(11) 6256-1308
4,DNI,35399142,EMAIL,vanesa.biz@gmail.com


In [73]:
data1 = data["TIPO_DOC"]
data2 = proveedor["TIPO_DOC"]
data1 in data2

TypeError: unhashable type: 'Series'